<a href="https://colab.research.google.com/github/Matthew-Edgin/Matthew-Edgin.github.io/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. First, make some data preprocessing to clean up the data. You can use your solution to the assignment of the data preprocessing checkpoint of this module.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
import spacy
import re
!pip install chatterbot
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer
from chatterbot.conversation import Statement
import random

import warnings
warnings.filterwarnings(action="ignore")

!python -m spacy download en;

     |████████████████████████████████| 71kB 4.3MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'cornell_movie_dialogs'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

dialogs_df = pd.read_sql_query('select * from dialogs',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))

# below is necessary to avoid memory error of SpaCy
nlp.max_length = 20000000

# all the processing work is done below, so it may take a while
dialogs_doc = nlp(" ".join(dialogs_df.dialogs))

In [ ]:
dialog_sents = [sent.text for sent in dialogs_doc.sents if len(sent.text) > 1]

## 2. Develop a chatbot using this corpus. In doing this, you're free to choose a chatbot development libary like ChatterBot or to write your own code from scratch.

In [ ]:
GREETING_INPUTS = ["hello", "hi", "greetings", "what's up","hey"]
GREETING_RESPONSES = ["hello", "hi", "hey", "hi there"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [ ]:
# create a chatbot
chatbot = ChatBot('Dialogs')
# this is to remove the accumulated knowledge base
chatbot.storage.drop()

# create a new trainer for the chatbot
trainer = ListTrainer(chatbot)

# train the chatbot
trainer.train(dialog_sents)

List Trainer: [####################] 100%


Although the cell ran sucessfully it took over an hour to run even on a higher ram and TPU settings.

## 3. Make a conversation with your chatbot and discuss the strengths and weaknesses of it.

In [ ]:
import random

In [ ]:
print("Cornell: I will try to respond you reasonably. If you want to exit, type bye please.")

# below is the chatting
while True:
    
    user_input = input("User: ")
    user_input=user_input.lower()
    
    if(user_input!='bye'):
        if(user_input == 'thanks' or user_input == 'thank you'):
            break
            print("Cornell: You're welcome.")
        else:
            if(greeting(user_input) != None):
                print("Cornell: " + greeting(user_input))
            else:
                print("Cornell: ", end = "")
                print(chatbot.get_response(user_input))
    else:
        print("Cornell: Bye! It was a great chat.")
        break

Cornell: I will try to respond you reasonably. If you want to exit, type bye please.
User: hello
Cornell: hey
User: how are you
Cornell: Takin' a bath.
User: why a bath?
Cornell: You're welcome.
User: your an odd fellow
Cornell: I think I'm right in saying that, aren't I, Court Composer?
User: Can you dance?
Cornell: Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.
User: Oh no!
Cornell: NO...!
User: bye
Cornell: Bye! It was a great chat.


Not bad, right? We can really hear the Hollywood jargon in the conversation. As expected, our bot carries the colors and the biases in the movies!